Goodness-of-fit — пример: тест на соответствие распределению Пуассона

In [1]:
import numpy as np
from scipy.stats import chisquare, poisson
import math

np.random.seed(0)
n = 300
lam_true = 2.3
data = np.random.poisson(lam_true, size=n)

# группируем по значениям 0,1,...,max и объединяем хвост
max_k = data.max()
bins = list(range(0, max_k)) + [max_k+1]
values, counts = np.unique(data, return_counts=True)

# оценка lambda
lam_hat = data.mean()
probs = [poisson.pmf(k, lam_hat) for k in range(0, max_k)]
prob_tail = 1 - sum(probs)
probs.append(prob_tail)

E = np.array([p * n for p in probs])
O = np.array([counts[k] if k in values else 0 for k in range(0, max_k)] + [sum(counts[values>=max_k])])

# df = k_bins - 1 - m (m=1 параметр)
k_bins = len(O)
df = k_bins - 1 - 1

chi2_stat = ((O - E)**2 / E).sum()
from scipy.stats import chi2
pval = 1 - chi2.cdf(chi2_stat, df)
print("O:", O)
print("E:", np.round(E,2))
print("chi2 stat:", chi2_stat, "df:", df, "p-value:", pval)

O: [25 82 79 54 32 16  7  3  0  2]
E: [29.98 69.05 79.52 61.05 35.16 16.2   6.22  2.05  0.59  0.19]
chi2 stat: 22.260341083701125 df: 8 p-value: 0.004455742458216783


Contingency table — тест независимости

In [3]:
import numpy as np
from scipy.stats import chi2_contingency

table = np.array([[30, 20],
                  [10, 40],
                  [25, 15]])
chi2, p, dof, expected = chi2_contingency(table, correction=False)
print("chi2:", chi2, "p:", p, "dof:", dof)
print("expected:\n", expected)
#Монте-Карло p-value для малых ожидаемых частот
def monte_carlo_chi2_pvalue(observed_counts, expected_probs, B=5000, seed=0):
    rng = np.random.default_rng(seed)
    n = observed_counts.sum()
    obs_stat = (((observed_counts - expected_probs*n)**2) / (expected_probs*n)).sum()
    stats = np.empty(B)
    for i in range(B):
        samp = rng.multinomial(n, expected_probs)
        stats[i] = (((samp - expected_probs*n)**2) / (expected_probs*n)).sum()
    return (stats >= obs_stat).mean(), obs_stat

p_mc, obs_stat = monte_carlo_chi2_pvalue(O, np.array(probs), B=5000)
print("Monte Carlo p-value:", p_mc, "obs stat:", obs_stat)

chi2: 21.897435897435894 p: 1.7580540012965576e-05 dof: 2
expected:
 [[23.21428571 26.78571429]
 [23.21428571 26.78571429]
 [18.57142857 21.42857143]]
Monte Carlo p-value: 0.0212 obs stat: 22.260341083701125


Упражнение 1

In [10]:
import numpy as np
from scipy import stats

np.random.seed(42)
data = np.random.poisson(3, 200)

# Частоты
obs = np.bincount(data)

# Ожидаемые для Pois(3)
exp = [200 * stats.poisson.pmf(i, 3) for i in range(len(obs))]

# Находим первое место, где E < 5
cut_idx = next(i for i, e in enumerate(exp) if e < 5)

# Объединяем
obs_combined = list(obs[:cut_idx]) + [sum(obs[cut_idx:])]
exp_combined = list(exp[:cut_idx]) + [sum(exp[cut_idx:])]

# χ²-тест
chi2 = sum((o - e)**2 / e for o, e in zip(obs_combined, exp_combined))
df = len(obs_combined) - 1  # λ задан, не оценивается!
p_value = 1 - stats.chi2.cdf(chi2, df)

print(f"Ответ: {p_value:.1f}")

Ответ: 0.4


Упражнение 3

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats

np.random.seed(61)

groups = np.random.choice([0, 1, 2], size = 300)
categories = np.random.choice([0, 1, 2], size = 300)

table = pd.crosstab(groups, categories).values

# Выполнение теста хи-квадрат на независимость
chi2, p, dof, expected = stats.chi2_contingency(table)

print(f"p-value: {p}")

p-value: 0.02676610462759972


: 

Упражнение 4

In [35]:
import numpy as np
from scipy.stats import chisquare

np.random.seed(42)

# Генерация категориальных данных: 8 категорий, n=50
categories = np.random.choice(8, size=50)
observed = np.bincount(categories, minlength=8)

print("Наблюдаемые частоты:", observed)

# Хи-квадрат тест напрямую
chi2_stat, p_value = chisquare(observed)
print(f"\nПрямой тест: χ²={chi2_stat:.2f}, p-value={p_value:.4f}")

# Объединение категорий так, чтобы E_i ≥ 5
expected = 50 / 8  # 6.25 > 5, но проверим наблюдаемые частоты

# Объединяем категории с малыми наблюдаемыми частотами
# Простое объединение: группируем соседние категории
observed_combined = []
temp_sum = 0
for i, obs in enumerate(observed):
    temp_sum += obs
    if temp_sum >= 5 or i == len(observed)-1:
        observed_combined.append(temp_sum)
        temp_sum = 0

observed_combined = np.array(observed_combined)
print("\nОбъединенные частоты:", observed_combined)

# Новый хи-квадрат тест
chi2_stat_combined, p_value_combined = chisquare(observed_combined)
print(f"После объединения: χ²={chi2_stat_combined:.2f}, p-value={p_value_combined:.4f}")

# Сравнение
print(f"\nСравнение p-values: {p_value:.4f} vs {p_value_combined:.4f}")
print(f"p-value после объединения {'>' if p_value_combined > p_value else '<'} исходного")

Наблюдаемые частоты: [3 5 7 8 7 6 6 8]

Прямой тест: χ²=3.12, p-value=0.8737

Объединенные частоты: [8 7 8 7 6 6 8]
После объединения: χ²=0.68, p-value=0.9949

Сравнение p-values: 0.8737 vs 0.9949
p-value после объединения > исходного


Упражнение 5

In [36]:
import numpy as np
from scipy.stats import fisher_exact

# Таблица 2x2
table = np.array([[1, 4],
                  [0, 5]])

# Точный тест Фишера, двусторонняя альтернатива
odds_ratio, p_value = fisher_exact(table, alternative='two-sided')

# Округление до 3 знаков
p_value_rounded = round(p_value, 3)

print(f"p-value: {p_value}")
print(f"p-value (округлено до 3 знаков): {p_value_rounded}")

p-value: 1.0
p-value (округлено до 3 знаков): 1.0


### Критерий Пирсона для проверки параметрических гипотез

#### Проверка Пуассона с неизвестным лямбда
Сгенерируем выборку из пуассоновского распределения, оценим лямбда по выборке, разобьём по значениям 0,1,2,… и объединим хвост, чтобы Ei>=5.

In [1]:
import numpy as np
from scipy.stats import poisson, chi2
np.random.seed(0)

def pearson_chi2_goodness(data, pmf_func, params, max_bin=None):
    n = len(data)
    if max_bin is None:
        max_bin = data.max()
    obs_counts = np.array([np.sum(data==k) for k in range(0, max_bin)] + [np.sum(data>=max_bin)])
    probs = np.array([pmf_func(k, *params) for k in range(0, max_bin)])
    probs = np.append(probs, 1.0 - probs.sum())
    exp_counts = probs * n
    chi2_stat = ((obs_counts - exp_counts)**2 / exp_counts).sum()
    return chi2_stat, obs_counts, exp_counts

n = 300
lambda_true = 2.3
data = np.random.poisson(lambda_true, size=n)
lambda_hat = data.mean()
chi2_stat, O, E = pearson_chi2_goodness(data, poisson.pmf, (lambda_hat,), max_bin=6)
k_bins = len(O)
l = 1
df = k_bins - 1 - l
pval = 1 - chi2.cdf(chi2_stat, df)
print("chi2:", chi2_stat, "df:", df, "p-value:", pval)
print("O:", O)
print("E:", np.round(E,2))

chi2: 5.32438865718315 df: 5 p-value: 0.37758660568067004
O: [25 82 79 54 32 16 12]
E: [29.98 69.05 79.52 61.05 35.16 16.2   9.05]


Наблюдаемые частоты (O) и ожидаемые частоты (E) достаточно близки
Различия между ними статистически не значимы
Распределение данных можно считать пуассоновским

#### Monte-Carlo p-value (когда асимптотика ненадёжна)
Если некоторые Ei малы или n невелико — имитируем распределение статистики под H0 условно на эта:

In [ ]:
def monte_carlo_pvalue(data, pmf_sampler, params_hat, max_bin, B=5000, seed=1):
    rng = np.random.default_rng(seed)
    n = len(data)
    chi2_obs, O_obs, E_obs = pearson_chi2_goodness(data, pmf_sampler.pmf, params_hat, max_bin)
    stats = np.empty(B)
    for b in range(B):
        sim = rng.poisson(lam=params_hat['lam'], size=n)
        chi2_sim, _, _ = pearson_chi2_goodness(sim, poisson.pmf, (params_hat['lam'],), max_bin)
        stats[b] = chi2_sim
    p_mc = (stats >= chi2_obs).mean()
    return p_mc, chi2_obs

params_hat = {'lam': lambda_hat}
p_mc, chi2_obs = monte_carlo_pvalue(data, poisson, params_hat, max_bin=6, B=2000, seed=2)
print("Monte-Carlo p-value:", p_mc, "chi2_obs:", chi2_obs)

Упражнение 1

In [13]:
import numpy as np
from scipy.stats import poisson, chi2
np.random.seed(42)

def pearson_chi2_goodness(data, pmf_func, params, max_bin=None):
    n = len(data)
    if max_bin is None:
        max_bin = data.max()
    obs_counts = np.array([np.sum(data==k) for k in range(0, max_bin)] + [np.sum(data>=max_bin)])
    probs = np.array([pmf_func(k, *params) for k in range(0, max_bin)])
    probs = np.append(probs, 1.0 - probs.sum())
    exp_counts = probs * n
    chi2_stat = ((obs_counts - exp_counts)**2 / exp_counts).sum()
    return chi2_stat, obs_counts, exp_counts

n = 200
lambda_true = 3
data = np.random.poisson(lambda_true, size=n)
lambda_hat = data.mean()
chi2_stat, O, E = pearson_chi2_goodness(data, poisson.pmf, (lambda_hat,), max_bin=6)
k_bins = len(O)
l = 1
df = k_bins - 1 -l
pval = 1 - chi2.cdf(chi2_stat, df)
print("chi2:", chi2_stat, "df:", df, "p-value:", pval)
print("O:", O)
print("E:", np.round(E,2))

chi2: 5.671421302837369 df: 5 p-value: 0.33951472741619815
O: [14 37 40 44 25 24 16]
E: [11.34 32.55 46.7  44.68 32.06 18.4  14.28]


In [16]:
import numpy as np
from scipy.stats import poisson, chi2

np.random.seed(42)

def monte_carlo_pvalue(data, pmf_sampler, lambda_hat, max_bin, B=3000, seed=42):
    rng = np.random.default_rng(seed)
    n = len(data)
    # Используем кортеж параметров, а не словарь
    chi2_obs, O_obs, E_obs = pearson_chi2_goodness(data, pmf_sampler.pmf, (lambda_hat,), max_bin)
    stats = np.empty(B)
    
    for b in range(B):
        # Генерация данных под нулевой гипотезой
        sim = rng.poisson(lam=lambda_hat, size=n)
        chi2_sim, _, _ = pearson_chi2_goodness(sim, poisson.pmf, (lambda_hat,), max_bin)
        stats[b] = chi2_sim
    
    p_mc = (stats >= chi2_obs).mean()
    return p_mc, chi2_obs

# Основной код
n = 200
lambda_true = 3
data = np.random.poisson(lambda_true, size=n)

# Оценка λ
lambda_hat = data.mean()

# Хи-квадрат тест с группировкой 0..5, хвост ≥6
max_bin = 5
chi2_stat, O, E = pearson_chi2_goodness(data, poisson.pmf, (lambda_hat,), max_bin=max_bin)

# Степени свободы
k_bins = len(O)
df = k_bins - 1 - 1  # минус 1 (сумма) и минус 1 (оцененный параметр)
pval = 1 - chi2.cdf(chi2_stat, df)
pval_rounded = round(pval, 1)

# Монте-Карло p-value
p_mc, chi2_obs = monte_carlo_pvalue(data, poisson, lambda_hat, max_bin=5, B=3000, seed=42)
print(f"\nМонте-Карло p-value (B=3000): {p_mc:.4f}")
print(f"Наблюдаемое χ²: {chi2_obs:.3f}")


Монте-Карло p-value (B=3000): 0.3697
Наблюдаемое χ²: 5.400


Упражнение 5

In [19]:
import numpy as np

np.random.seed(14)

# Параметры: m = 60 наблюдений, k = 10 категорий
m = 60
k = 10

# Генерируем равновероятные категориальные данные
data = np.random.choice(k, size=m)


In [20]:
# Частоты по категориям (наблюдаемые)
observed = np.bincount(data, minlength=k)

# Хи-квадрат тест для равномерного распределения
expected_uniform = np.full(k, m/k)  # 60/10 = 6

# 1. Исходный хи-квадрат тест
chi2_stat_original = np.sum((observed - expected_uniform)**2 / expected_uniform)
df_original = k - 1
p_value_original = 1 - chi2.cdf(chi2_stat_original, df_original)

print(f"Исходный p-value: {p_value_original:.4f}")

# 2. Объединяем категории, чтобы все E_i ≥ 5
observed_combined = observed.copy()
expected_combined = expected_uniform.copy()

i = 0
while i < len(observed_combined):
    if expected_combined[i] < 5:
        # Объединяем с соседней категорией
        if i == len(observed_combined) - 1:
            # Последняя категория - объединяем с предыдущей
            observed_combined[i-1] += observed_combined[i]
            expected_combined[i-1] += expected_combined[i]
            observed_combined = np.delete(observed_combined, i)
            expected_combined = np.delete(expected_combined, i)
        else:
            # Объединяем со следующей категорией
            observed_combined[i+1] += observed_combined[i]
            expected_combined[i+1] += expected_combined[i]
            observed_combined = np.delete(observed_combined, i)
            expected_combined = np.delete(expected_combined, i)
    else:
        i += 1

# Хи-квадрат после объединения
chi2_stat_combined = np.sum((observed_combined - expected_combined)**2 / expected_combined)
df_combined = len(observed_combined) - 1
p_value_combined = 1 - chi2.cdf(chi2_stat_combined, df_combined)

print("\nПосле объединения категорий:")
print(f"P-value: {p_value_combined:.4f}")

# 3. Monte-Carlo оценка p-value с B = 3000
B = 3000
extreme_count = 0
rng = np.random.default_rng(42)  # отдельный seed для MC

for _ in range(B):
    # Генерируем данные под нулевой гипотезой (равномерное распределение)
    sim_data = rng.choice(k, size=m)
    sim_observed = np.bincount(sim_data, minlength=k)
    
    # Объединяем так же, как и в основном тесте
    sim_obs_combined = sim_observed.copy()
    sim_exp_combined = expected_uniform.copy()
    
    i = 0
    while i < len(sim_obs_combined):
        if sim_exp_combined[i] < 5:
            if i == len(sim_obs_combined) - 1:
                sim_obs_combined[i-1] += sim_obs_combined[i]
                sim_exp_combined[i-1] += sim_exp_combined[i]
                sim_obs_combined = np.delete(sim_obs_combined, i)
                sim_exp_combined = np.delete(sim_exp_combined, i)
            else:
                sim_obs_combined[i+1] += sim_obs_combined[i]
                sim_exp_combined[i+1] += sim_exp_combined[i]
                sim_obs_combined = np.delete(sim_obs_combined, i)
                sim_exp_combined = np.delete(sim_exp_combined, i)
        else:
            i += 1
    
    # Хи-квадрат для симулированных данных
    sim_chi2 = np.sum((sim_obs_combined - sim_exp_combined)**2 / sim_exp_combined)
    
    if sim_chi2 >= chi2_stat_combined:
        extreme_count += 1

mc_p_value = extreme_count / B
mc_p_value_rounded = round(mc_p_value, 1)

print(f"\nMonte-Carlo p-value (B={B}): {mc_p_value:.4f}")
print(f"Monte-Carlo p-value (округлено до 1 знака): {mc_p_value_rounded}")

Исходный p-value: 0.7061

После объединения категорий:
P-value: 0.7061

Monte-Carlo p-value (B=3000): 0.7300
Monte-Carlo p-value (округлено до 1 знака): 0.7
